In [1]:
import kagglehub
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

2025-06-05 18:46:08.667589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749138368.686286  301642 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749138368.692364  301642 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749138368.707218  301642 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749138368.707243  301642 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749138368.707244  301642 computation_placer.cc:177] computation placer alr

In [2]:
# Download latest version
path = kagglehub.dataset_download("danofer/sarcasm")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/danofer/sarcasm/versions/4


In [3]:
dataset = pd.read_csv(path + '/train-balanced-sarcasm.csv')
dataset.head(3)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.


In [4]:
dataset = dataset[['label', 'comment']]
dataset = dataset.sample(n=10000)
dataset.info(), dataset.head(3), dataset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 508230 to 4618
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    10000 non-null  int64 
 1   comment  9999 non-null   object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


(None,
         label                                            comment
 508230      0                                               Cool
 110306      1  or at least put them all in the same place so ...
 581170      0                                        yay 150mil!,
 label      0
 comment    1
 dtype: int64)

In [5]:
dataset.dropna(inplace=True)
dataset.isna().sum()

label      0
comment    0
dtype: int64

In [6]:
dataset['comment'] = dataset['comment'].apply(lambda comment: comment.lower())
dataset['comment'] = dataset['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
dataset.sample(10)

,label,comment
652095,0,if it has dna or rna it has protein
387112,1,dont worry they have gear score brackets
485395,1,lunatone and solrock confirmed new legendaries
862310,1,its not like ajax and barcelona have a history...
548128,1,it doesnt matter you fucking shitlord
321369,1,baka yung ipapa vp nila si mocha uson
136345,0,yeah ended up winning after that
444313,0,just the first if you are using serato the ot...
126215,1,you forgot the
564257,0,except bernie sanders got like x more coverage


In [7]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [8]:
def tokenize_text(text):
    return tokenizer(
         text.tolist(),
         max_length = 100,
         truncation = True,
         padding = 'max_length',
         return_tensors = 'np'
    )

tokenized_comments = tokenize_text(dataset['comment'])
tokenized_comments

{'input_ids': array([[  101,  4658,   102, ...,     0,     0,     0],
       [  101,  2030,  2012, ...,     0,     0,     0],
       [  101,  8038,  2100, ...,     0,     0,     0],
       ...,
       [  101, 13659,  1999, ...,     0,     0,     0],
       [  101,  2092,  2008, ...,     0,     0,     0],
       [  101,  2821,  6300, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [18]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
model = tf.keras.Sequential([
    bert_model,
    tf.keras.layers.Lambda(lambda bert_output: bert_output.pooler_output, name="extract_pooler_output"),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [19]:
from sklearn.model_selection import train_test_split


labels_tf = tf.constant(dataset['label'].values)
indices = np.arange(dataset.shape[0])
train_indices, test_indices = train_test_split(
    indices,
    test_size=0.2,
    stratify=dataset['label'].values
)

X_train = {
        'input_ids': tf.gather(tokenized_comments['input_ids'], train_indices),
        'attention_mask': tf.gather(tokenized_comments['attention_mask'], train_indices)
    }

X_test = {
    'input_ids': tf.gather(tokenized_comments['input_ids'], test_indices),
    'attention_mask': tf.gather(tokenized_comments['attention_mask'], test_indices)
}

y_train = tf.gather(labels_tf, train_indices)
y_test = tf.gather(labels_tf, test_indices)

model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
500/500 [==============================] - 149s 189ms/step - loss: 0.6688 - accuracy: 0.5738
Epoch 2/10
500/500 [==============================] - 81s 162ms/step - loss: 0.5876 - accuracy: 0.7007
Epoch 3/10
500/500 [==============================] - 81s 163ms/step - loss: 0.4488 - accuracy: 0.7955
Epoch 4/10
500/500 [==============================] - 81s 162ms/step - loss: 0.2550 - accuracy: 0.9019
Epoch 5/10
500/500 [==============================] - 81s 163ms/step - loss: 0.1257 - accuracy: 0.9587
Epoch 6/10
500/500 [==============================] - 81s 161ms/step - loss: 0.0751 - accuracy: 0.9751
Epoch 7/10
500/500 [==============================] - 81s 163ms/step - loss: 0.0515 - accuracy: 0.9829
Epoch 8/10
500/500 [==============================] - 81s 162ms/step - loss: 0.0449 - accuracy: 0.9847
Epoch 9/10
500/500 [==============================] - 80s 160ms/step - loss: 0.0403 - accuracy: 0.9876
Epoch 10/10
500/500 [==============================] - 81s 162ms/step - 

In [20]:
y_pred_trained = model.predict(X_train)
y_pred_test = model.predict(X_test)

63/63 [==============================] - 7s 115ms/step


In [21]:
max(y_pred_trained), min(y_pred_trained), max(y_pred_test), min(y_pred_test)

(array([0.99998665], dtype=float32),
 array([1.5665855e-06], dtype=float32),
 array([0.99998665], dtype=float32),
 array([1.4852164e-06], dtype=float32))

In [22]:
y_pred_train_results = (y_pred_trained > 0.5).astype(int).flatten()
y_pred_test_results = (y_pred_test > 0.5).astype(int).flatten()

full_pred = np.empty(len(dataset), dtype=int)

full_pred[train_indices] = y_pred_train_results
full_pred[test_indices] = y_pred_test_results
    
dataset['predicted'] = full_pred
dataset.sample(20)

,label,comment,predicted
608726,0,yes,0
780451,1,the talk about joe kelly winning a cy young ha...,1
443693,0,weeb gachi dansgame,0
220293,1,this movie will literally only have smple in it,1
955985,1,hahahahaha great photoshoot op,0
487446,1,youre the best security guard ever,1
628212,1,but everyone mains falcon so it doesnt count,1
772646,0,whats your vector victor,0
56233,1,says the guy who isnt married,1
263774,0,this has to be the most tumblr title i have ev...,0


In [23]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred_test_results)
print(report)

              precision    recall  f1-score   support

           0       0.68      0.67      0.68      1006
           1       0.67      0.68      0.68       994

    accuracy                           0.68      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.68      0.68      0.68      2000

